In [7]:
#from pathlib import Path
from PIL import Image
import numpy as np
import glob
from facenet_models import FacenetModel

In [8]:
model = FacenetModel()
'''
for filename in glob.glob(r'pics\train\*.jpg'):
    pic = Image.open(filename)
    pic_arr = np.array(pic)
    boxes, probabilities, landmarks = model.detect(pic_arr)
    descriptors = model.compute_descriptors(pic_arr, boxes)

#pic = Image.open(r'pics/test/Will_Smith_test.jpg')  
#pic_arr = np.array(pic)'''

"\nfor filename in glob.glob(r'pics\train\\*.jpg'):\n    pic = Image.open(filename)\n    pic_arr = np.array(pic)\n    boxes, probabilities, landmarks = model.detect(pic_arr)\n    descriptors = model.compute_descriptors(pic_arr, boxes)\n\n#pic = Image.open(r'pics/test/Will_Smith_test.jpg')  \n#pic_arr = np.array(pic)"

In [9]:
for filename in glob.glob(r'pics\train\*.jpg'):
    print(filename)
    print(filename[11:-6])
    pic = Image.open(filename)
    pic_arr = np.array(pic)
    boxes, probabilities, landmarks = model.detect(pic_arr)
    descriptors = model.compute_descriptors(pic_arr, boxes)

pics\train\Angelina_Jolie_1.jpg
Angelina_Jolie
pics\train\Angelina_Jolie_2.jpg
Angelina_Jolie
pics\train\Angelina_Jolie_3.jpg
Angelina_Jolie
pics\train\Scarlett_Johansson_1.jpg
Scarlett_Johansson
pics\train\Scarlett_Johansson_2.jpg
Scarlett_Johansson
pics\train\Scarlett_Johansson_3.jpg
Scarlett_Johansson
pics\train\Tom_Cruise_1.jpg
Tom_Cruise
pics\train\Tom_Cruise_2.jpg
Tom_Cruise
pics\train\Tom_Cruise_3.jpg
Tom_Cruise
pics\train\Will_Smith_1.jpg
Will_Smith
pics\train\Will_Smith_2.jpg
Will_Smith
pics\train\Will_Smith_3.jpg
Will_Smith
